In [1]:
import requests
import csv
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [2]:
def movie_scrapper(url, m_results):
    results = requests.get(url)

    if results.status_code != 200:
        return f'status failed with {results.status_code}'
    else:
        # use BeautifulSoup to parse the contents of the request
        soup = BeautifulSoup(results.content, "html.parser")

        movie_results = soup.find(id="table")
        # print(movie_results)
        if movie_results:
            table = movie_results.find('table', class_="a-bordered a-horizontal-stripes a-size-base a-span12 mojo-body-table mojo-table-annotated")
            if table:
                rows = table.find_all("tr")[1:]
                for row in rows:
                    cells = row.find_all("td")
                    if len(cells) >= 7: #added check to make sure there are enough cells to prevent index errors.
                        row_data = {
                            "Movie_Rank": cells[0].text.strip(),
                            "Movie_Name": cells[1].text.strip(),
                            "Worldwide_Gross": cells[2].text.strip(),
                            "Domestic_Gross": cells[3].text.strip(),
                            "Domestic_Percent": cells[4].text.strip(),
                            "Foreign_Gross": cells[5].text.strip(),
                            "Foreign_Percent": cells[6].text.strip(),
                        }
                        # print(row_data)
                        m_results.append(row_data)
                    else:
                        print("Row has insufficient data.")
            else:
                print("Inner table not found.")
        else:
            print("Table element not found.")
                

In [3]:
target_url = 'https://www.boxofficemojo.com/year/world/'
years = ["2000/","2001/","2002/","2003/","2004/","2005/","2006/","2007/","2008/","2009/","2010/","2011/","2012/","2013/","2014/","2015/","2016/","2017/","2018/","2019/","2020/","2021/","2022/","2023/","2024/","2025/"]
m_res = []
for year in years:
    url = target_url + year
    # print(url)
    movie_scrapper(url,m_res)
df= pd.DataFrame(m_res)
print(df.head())
# print(m_res)

  Movie_Rank              Movie_Name Worldwide_Gross Domestic_Gross  \
0          1  Mission: Impossible II    $546,388,108   $215,409,889   
1          2               Gladiator    $460,583,960   $187,705,427   
2          3               Cast Away    $429,632,142   $233,632,142   
3          4         What Women Want    $374,111,707   $182,811,707   
4          5                Dinosaur    $349,822,765   $137,748,063   

  Domestic_Percent Foreign_Gross Foreign_Percent  
0            39.4%  $330,978,219           60.6%  
1            40.8%  $272,878,533           59.2%  
2            54.4%  $196,000,000           45.6%  
3            48.9%  $191,300,000           51.1%  
4            39.4%  $212,074,702           60.6%  


In [4]:
df.to_csv("movie_dataset_grossing.csv", index=False)

In [5]:
df = pd.read_csv("movie_dataset_grossing.csv")

In [6]:
df = df.replace('-',np.nan)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5200 entries, 0 to 5199
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Movie_Rank        5200 non-null   int64 
 1   Movie_Name        5200 non-null   object
 2   Worldwide_Gross   5200 non-null   object
 3   Domestic_Gross    3879 non-null   object
 4   Domestic_Percent  3879 non-null   object
 5   Foreign_Gross     5147 non-null   object
 6   Foreign_Percent   5147 non-null   object
dtypes: int64(1), object(6)
memory usage: 284.5+ KB


# Cleaning of values

Converting Domestic_Gross to digits and replacing missing values with mean

In [8]:
df['Domestic_Gross'] = pd.to_numeric(df['Domestic_Gross'].str.replace('$', '').str.replace(',', ''), errors='coerce')
# print(df['Domestic_Gross'])


In [9]:
mean = df['Domestic_Gross'].mean()
print(mean)

57982996.937870584


In [10]:
df['Domestic_Gross'] = df['Domestic_Gross'].fillna(mean)
print(df['Domestic_Gross'])

0       2.154099e+08
1       1.877054e+08
2       2.336321e+08
3       1.828117e+08
4       1.377481e+08
            ...     
5195    5.798300e+07
5196    5.798300e+07
5197    1.922230e+05
5198    5.798300e+07
5199    2.162400e+04
Name: Domestic_Gross, Length: 5200, dtype: float64


In [11]:
df['Domestic_Gross'] = df['Domestic_Gross'].round(2)

In [12]:
print(df['Domestic_Gross'])

0       2.154099e+08
1       1.877054e+08
2       2.336321e+08
3       1.828117e+08
4       1.377481e+08
            ...     
5195    5.798300e+07
5196    5.798300e+07
5197    1.922230e+05
5198    5.798300e+07
5199    2.162400e+04
Name: Domestic_Gross, Length: 5200, dtype: float64


Converting Domestic_Percent to digits and replacing missing values with mean

In [13]:
df['Domestic_Percent'] = pd.to_numeric(df['Domestic_Percent'].str.replace('%', '').str.replace(',', ''), errors='coerce')
print(df['Domestic_Percent'])

0       39.4
1       40.8
2       54.4
3       48.9
4       39.4
        ... 
5195     NaN
5196     NaN
5197    45.6
5198     NaN
5199     5.3
Name: Domestic_Percent, Length: 5200, dtype: float64


In [14]:
mean = df['Domestic_Percent'].mean()
print(mean)

47.41307752545028


In [15]:
df['Domestic_Percent'] = df['Domestic_Percent'].fillna(mean)
print(df['Domestic_Percent'])

0       39.400000
1       40.800000
2       54.400000
3       48.900000
4       39.400000
          ...    
5195    47.413078
5196    47.413078
5197    45.600000
5198    47.413078
5199     5.300000
Name: Domestic_Percent, Length: 5200, dtype: float64


In [16]:
df['Domestic_Percent'] = df['Domestic_Percent'].round(2)
print(df['Domestic_Percent'])

0       39.40
1       40.80
2       54.40
3       48.90
4       39.40
        ...  
5195    47.41
5196    47.41
5197    45.60
5198    47.41
5199     5.30
Name: Domestic_Percent, Length: 5200, dtype: float64


Converting Foreign_Gross to digits and replacing missing values with mean

In [17]:
df['Foreign_Gross'] = pd.to_numeric(df['Foreign_Gross'].str.replace('$', '').str.replace(',', ''), errors='coerce')
print(df['Foreign_Gross'])

0       330978219.0
1       272878533.0
2       196000000.0
3       191300000.0
4       212074702.0
           ...     
5195       422935.0
5196       422635.0
5197       229013.0
5198       413002.0
5199       385162.0
Name: Foreign_Gross, Length: 5200, dtype: float64


In [18]:
mean = df['Foreign_Gross'].mean()
print(mean)

73995924.07693802


In [19]:
df['Foreign_Gross'] = df['Foreign_Gross'].fillna(mean)
print(df['Foreign_Gross'])

0       330978219.0
1       272878533.0
2       196000000.0
3       191300000.0
4       212074702.0
           ...     
5195       422935.0
5196       422635.0
5197       229013.0
5198       413002.0
5199       385162.0
Name: Foreign_Gross, Length: 5200, dtype: float64


In [20]:
df['Foreign_Gross'] = df['Foreign_Gross'].round(2)
print(df['Foreign_Gross'])

0       330978219.0
1       272878533.0
2       196000000.0
3       191300000.0
4       212074702.0
           ...     
5195       422935.0
5196       422635.0
5197       229013.0
5198       413002.0
5199       385162.0
Name: Foreign_Gross, Length: 5200, dtype: float64


Converting Foreign_Percentage to digits and replacing missing values with mean

In [21]:
df['Foreign_Percent'] = pd.to_numeric(df['Foreign_Percent'].str.replace('%', '').str.replace(',', ''), errors='coerce')

In [22]:
mean = df['Foreign_Percent'].mean()
print(mean)

65.85157648890619


In [23]:
df['Foreign_Percent'] = df['Foreign_Percent'].fillna(mean)
print(df['Foreign_Percent'])

0        60.6
1        59.2
2        45.6
3        51.1
4        60.6
        ...  
5195    100.0
5196    100.0
5197     54.4
5198    100.0
5199     94.7
Name: Foreign_Percent, Length: 5200, dtype: float64


In [24]:
df['Foreign_Percent'] = df['Foreign_Percent'].round(2)
print(df['Foreign_Percent'])

0        60.6
1        59.2
2        45.6
3        51.1
4        60.6
        ...  
5195    100.0
5196    100.0
5197     54.4
5198    100.0
5199     94.7
Name: Foreign_Percent, Length: 5200, dtype: float64


In [25]:
df.to_csv("movie_dataset_grossing_cleaned.csv", index=False)

In [26]:
movies_df = pd.read_csv("movies.csv")

def extract_year(title):
    try:
        return int(title[-5:-1])
    except:
        return None
    
movies_df['Year'] = movies_df['title'].apply(extract_year)
print(movies_df['Year'])

0        1995.0
1        1995.0
2        1995.0
3        1995.0
4        1995.0
          ...  
86532    2021.0
86533    2021.0
86534    1973.0
86535    2023.0
86536    2021.0
Name: Year, Length: 86537, dtype: float64


In [35]:
movies_df['title'] =  movies_df['title'].str[:-7]
print(movies_df)

       movieId                                 title  \
0            1                                    To   
1            2                                         
2            3                             Grumpier    
3            4                            Waiting to   
4            5                  Father of the Bride    
...        ...                                   ...   
86532   288967                State of Siege: Temple   
86533   288971                                  Ouij   
86534   288975    The Men Who Made the Movies: Howar   
86535   288977                     Skinford: Death S   
86536   288983  UNZIPPED: An Autopsy of American Ine   

                                            genres    Year  
0      Adventure|Animation|Children|Comedy|Fantasy  1995.0  
1                       Adventure|Children|Fantasy  1995.0  
2                                   Comedy|Romance  1995.0  
3                             Comedy|Drama|Romance  1995.0  
4                     

In [29]:
df['Movie_Name'] =  df['Movie_Name'].str.strip()
print(df['Movie_Name'])

0       Mission: Impossible II
1                    Gladiator
2                    Cast Away
3              What Women Want
4                     Dinosaur
                 ...          
5195                    Dogulu
5196                      Spit
5197               Inheritance
5198                  Bromance
5199                     Putin
Name: Movie_Name, Length: 5200, dtype: object


In [36]:
import re

def clean_title(title):
    title = title.lower()
    title = re.sub(r'[^\w\s]', '', title)  # Remove punctuation
    return title.strip()

movies_df['title'] = movies_df['title'].apply(clean_title)

In [ ]:
movies_df['cleaned_title'] = movies_df['title'].apply(clean_title)

df['cleaned_movie_name'] = df['Movie_Name'].apply(clean_title) 



In [42]:
merged_df = pd.merge(df, movies_df, left_on='cleaned_movie_name', right_on='cleaned_title', how='left')
merged_df = merged_df.drop(['cleaned_movie_name','cleaned_title'], axis = 1)

In [45]:
print(merged_df)

      Movie_Rank              Movie_Name Worldwide_Gross  Domestic_Gross  \
0              1  Mission: Impossible II    $546,388,108    2.154099e+08   
1              2               Gladiator    $460,583,960    1.877054e+08   
2              3               Cast Away    $429,632,142    2.336321e+08   
3              4         What Women Want    $374,111,707    1.828117e+08   
4              5                Dinosaur    $349,822,765    1.377481e+08   
...          ...                     ...             ...             ...   
6114         196                  Dogulu        $422,935    5.798300e+07   
6115         197                    Spit        $422,635    5.798300e+07   
6116         198             Inheritance        $421,236    1.922230e+05   
6117         199                Bromance        $413,002    5.798300e+07   
6118         200                   Putin        $406,786    2.162400e+04   

      Domestic_Percent  Foreign_Gross  Foreign_Percent   movieId     title  \
0        

In [47]:
print(merged_df.notna().sum())

Movie_Rank          6119
Movie_Name          6119
Worldwide_Gross     6119
Domestic_Gross      6119
Domestic_Percent    6119
Foreign_Gross       6119
Foreign_Percent     6119
movieId             1281
title               1281
genres              1281
Year                1276
dtype: int64


In [ ]:
imdb_df = pd.read_csv("imdb_top_1000.csv")

